In [1]:
import __init__
from __init__ import DATA_PATH
from __init__ import PACKAGE_PATH
import numpy as np
import pandas as pd
import os
import matplotlib as mplt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_csv(os.path.join(DATA_PATH,'BOD_biclass_139rdkit.csv'))
data_139 = data.iloc[:,2:141]
data_train = data_139.values
data_train.shape

(1063, 139)

In [7]:
from dimreducer import DeepAutoEncoder
from cluster import KMeansLayer
from keras import Model

In [8]:
dims = [139,100,70]
autoencoder =  DeepAutoEncoder(dims)
autoencoder.build_autoencoder()
autoencoder.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 139)               0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 139)               556       
_________________________________________________________________
encoder_0 (Dense)            (None, 100)               14000     
_________________________________________________________________
embedding_layer (Dense)      (None, 70)                7070      
_________________________________________________________________
decoder_1 (Dense)            (None, 100)               7100      
_________________________________________________________________
decoder_output (Dense)       (None, 139)               14039     
Total params: 42,765
Trainable params: 42,487
Non-trainable params: 278
_________________________________________________________________


In [11]:
embeding   = autoencoder.model.get_layer(name='embedding_layer').output
encoder    = Model(inputs=autoencoder.model.input, outputs=embeding)
n_clusters = 2
clustering = KMeansLayer(n_clusters, name='clustering')(embeding)
model      = Model(inputs=autoencoder.model.input, outputs=[clustering,autoencoder.model.output])

In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 139)          0                                            
__________________________________________________________________________________________________
batch_normalization_3 (BatchNor (None, 139)          556         input[0][0]                      
__________________________________________________________________________________________________
encoder_0 (Dense)               (None, 100)          14000       batch_normalization_3[0][0]      
__________________________________________________________________________________________________
embedding_layer (Dense)         (None, 70)           7070        encoder_0[0][0]                  
__________________________________________________________________________________________________
decoder_1 

In [13]:
model.compile(loss={'clustering': 'kld', 'decoder_output': 'mse'},
              loss_weights=[0.1, 1],optimizer='adam')

In [46]:
from cluster import KMeans

In [53]:
kmeans_init = KMeans(n_clusters=2)
kmeans_init.bulid_kmeans_cluster()
kmeans_init.model.fit(encoder.predict(data_train))

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=1e-06, verbose=0)

In [54]:
y_pred = kmeans_init.model.labels_

In [57]:
model.get_layer(name='clustering').set_weights([kmeans_init.model.cluster_centers_])

In [94]:
q, _ = model.predict(data_train)

In [95]:
def target_distribution(q):  # target distribution P which enhances the discrimination of soft label Q
        weight = q ** 2 / q.sum(0)
        return (weight.T / weight.sum(1)).T

In [96]:
p = target_distribution(q)

In [97]:
y_pred = q.argmax(1)

In [98]:
loss = model.train_on_batch(x=data_train, y=[p,data_train])
loss

[1269.4034, 3.1364657e-06, 1269.4034]

In [99]:
model.metrics_names

['loss', 'clustering_loss', 'decoder_output_loss']

In [81]:
model.get_layer(name='clustering_layer').set_weights([kmeans_cluster.model.cluster_centers_])

In [15]:
loss = model.train_on_batch(data_train, data_train, epochs=150,
                            batch_size=256,shuffle=True)

ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), but instead got the following list of 1 arrays: [array([[232.08479224,   5.        ,   2.        , ...,   0.        ,
          0.        ,   0.        ],
       [ 90.03169405,   3.        ,   2.        , ...,   0.        ,
          0.        ,   ...

In [18]:
for i in range(8):
    print(i)
    i +=1

0
1
2
3
4
5
6
7
